In [ ]:
from math import tau

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import scipy.linalg as LA
import scipy.signal as ss

plt.style.use('seaborn-notebook')
np.random.seed(293710966)

# Esempio 3.14

In [ ]:
power_freq = 50
sampling_freq = 1130
time = np.arange(4096)
harmonic_numbers = np.array([1])
no_of_harmonics = harmonic_numbers.max()

noise = np.random.normal(0, 0.1, time.size)
omega = tau * power_freq / sampling_freq
signal = 300. * np.cos(omega * time) + noise

fig, ax = plt.subplots()
ax.plot(time[:200], signal[:200].real)
ax.set_xlabel('time [s]')
ax.set_ylabel('signal [V]')
fig.tight_layout()

In [ ]:
fastest_period = np.around(sampling_freq / power_freq).astype(int)
time_window = fastest_period * harmonic_numbers[-1]
data_size = signal.size - time_window + 1

windows = [signal[i : i + time_window] for i in range(data_size)]
data_matrix = np.vstack(windows)

## Matrice delle Basi Ortogonali
La matrice $U$ viene ricavata mediante Decomposizione ai Valori Singolari (SVD) della matrice dei campionamenti $R_v$

In [ ]:
_, singuar_values, unitary = LA.svd(data_matrix)
unitary_signal = unitary[:, :harmonic_numbers.size]
U1 = unitary_signal[:time_window-1]
U2 = unitary_signal[-time_window+1:]

La matrice $U$ viene poi partizionata estraendo le prime e ultime $\left( M - 1 \right)$ colonne.

## Stima per Minimi Quadrati (LS) di $\Psi$
$$
\hat{\Psi}_{LS} = \left( U_1^H U_1 \right)^{-1} U_1^H U_2
$$

In [ ]:
psi_ls = LA.inv(U1.T @ U1) @ U1.T @ U2
psi_ls.shape

## Stima delle Frequenze Armoniche
Calcolo degli autovalori di $\Psi$ coincidenti con i versori rotanti associati alle frequenze armoniche.

In [ ]:
LA.eigvals(psi_ls)

In [ ]:
est_omegas = np.log(LA.eigvals(psi_ls)).imag
est_freqs = est_omegas * sampling_freq / tau

fig, ax = plt.subplots()
ax.plot(power_freq * harmonic_numbers, 'o-', label='Harmonic Frequencies')
ax.plot(est_freqs, 'o-', label='Estimated Frequencies')
ax.set_ylabel('Frequency [Hz]')
ax.legend()
fig.tight_layout()